In [ ]:
import torch
import pyro
from pyro.distributions import *
#from collections import Counter
import pyro.infer
import pyro.optim
import pyro.util
pyro.enable_validation(True)

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import seaborn as sns
import tqdm

import numpy as np
import pandas as pd
import scipy.stats as stats
import scipy.special as special
import math

import os.path as path
from datetime import datetime
import json

import utils

import gc

In [ ]:
cpu = torch.cuda.is_available():

# TODO: set the GPU you want to use
gpu_n = 0

device = torch.device(f'cuda:{gpu_n}' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
def debug_tensors():
    for obj in gc.get_objects():
        if torch.is_tensor(obj) and obj.device == torch.device("cpu"):
            print(f"{obj.device}, {obj.dtype}, {obj.shape}")

debug_tensors()

In [ ]:
def save_rng_state(name):
    fn = name + '-' + datetime.today().isoformat() + '.state'
    state = pyro.util.get_rng_state()
    with open('rng-' + fn, 'w') as f:
        print(state, file=f)
    torch.save(state['torch'], 'torch-' + fn)

In [ ]:
torch.set_default_dtype(torch.float64)

# set random seeds
pyro.set_rng_seed(0)
#torch.set_deterministic(True)
#torch.set_num_threads(1)
#torch.set_num_interop_threads(1)

# fix the range of pitches we consider
fifth_range = 2*7                  # 2 diatonics
npcs = 2*fifth_range+1             # around C: Cbb to C## on LoF
utils.set_fifth_range(fifth_range) # used to make helper functions work correctly
nclusters = 14                      # Try clustering down to how many clusters?

# A Cognitive Model or Harmonic Types

## Model

A chord consists of a number of notes,
which are either generated as stable chord tones or as ornaments.
We model this process by distinguishing (for each chord type)
a distribution of chord tones and a distribution of ornaments.
For each generated note, a coin is flipped as to whether the tone is generated as a chord tone or ornament.
The pitch is then drawn from the corresponding distribution.
Since we don't always know the type of a note, we flip another coin to decide whether the type is observed or not (in which case `unknown` is emitted for both ornaments and chordtones).

Priors:
- choose $\vec\chi \sim \text{Dirichlet}(0.5, n_\text{harmonies})$
- choose $\lambda \sim \text{Gamma}(3,1)$
- choose $\theta_c \sim \text{Beta}(1,1)$ for each of $n_\text{clusters}$ clusters
- for each chord type $c$:
  - choose $\xi_c \sim \text{Categorical}(n_\text{clusters})$
  - choose $\vec\phi_{ct}^{(c)} \sim \text{Dirichlet}(0.5, n_\text{pitches})$
  - choose $\vec\phi_{or}^{(c)} \sim \text{Dirichlet}(0.5, n_\text{pitches})$

Generating a single chord (long version):
- choose $h \sim \text{Categorical}(\vec\chi)$
- choose $n \sim \text{Poisson}(\lambda) + 1$
- for each note $i \in 1, \ldots, n$:
  - choose $t_i \sim \text{Bernoulli}(\theta_{\xi_c})$
  - choose $p_i \sim \begin{cases}
                       \text{Categorical}(\vec\phi_{ct}^{(h)}) & \text{if } t_i = 1\\
                       \text{Categorical}(\vec\phi_{or}^{(h)}) & \text{if } t_i = 0
                     \end{cases}$
  - choose $o_i \sim \text{Bernoulli}(p_\text{obs})$
  - choose $ot_i = \begin{cases}
                     \text{'chordtone'} & \text{if } o_i = 1 \wedge p_i = 1\\
                     \text{'ornament'} & \text{if } o_i = 1 \wedge p_i = 0\\
                     \text{'unknown'} & \text{if } o_i = 0\\
                   \end{cases}$
- count $(p_i,ot_i)$ pairs

Generating a single chord (compact version)
- choose $h \sim \text{Catecorical}(\vec\chi)$
- choose $n \sim \text{Poisson}(\lambda) + 1$
- choose $n_{p,ot} \sim \text{Multinomial}(n, \vec\nu),$ where
  - $\nu_{ct} = p_\text{obs} \cdot \theta_{\xi_c} \cdot \vec\phi_{ct}^{(h)}$
  - $\nu_{or} = p_\text{obs} \cdot (1-\theta_{\xi_c}) \cdot \vec\phi_{or}^{(o)}$
  - $\nu_{uk} = (1-p_\text{obs}) \cdot \left( \theta_{\xi_c} \vec\phi_{ct}^{(h)} + (1-\theta_{\xi_c}) \vec\phi_{or}^{(h)} \right)$
  - $\nu = \text{concat}(\nu_{ct}, \nu_{or}, \nu_{uk})$

In [ ]:
def chord_model(npcs, nharmonies, cluster_assignment, data, pfx=None, subsamples=500, pobserve=0.5, **kwargs):
    if pfx is None or pfx == "":
        pfx = ""
    else:
        pfx += "_"
    nclusters = max(cluster_assignment) + 1
    # parameters priors:
    # distribution of the harmonies
    p_harmony = pyro.sample(f'{pfx}p_harmony', Dirichlet(0.5 * torch.ones(nharmonies, device=device)))
    # each cluster 
    with pyro.plate(f'{pfx}clusters', nclusters) as ind:
        # distribution of ornament probability
        p_is_chordtone = pyro.sample(f'{pfx}p_is_chordtone', Beta(torch.tensor(1, device=device),torch.tensor(1, device=device)))
    # distribution of notes in the harmonies
    with pyro.plate(f'{pfx}harmonies', nharmonies) as ind:
        #cluster = pyro.sample('cluster', Categorical(torch.ones(nclusters)))
        cluster = cluster_assignment[ind]
        p_is_chordtone_c = p_is_chordtone[cluster]
        #print(p_is_chordtone.shape)
        # distribution of notes per note type
        p_chordtones = pyro.sample(f'{pfx}p_chordtones', Dirichlet(0.5 * torch.ones(npcs, device=device)))
        p_ornaments  = pyro.sample(f'{pfx}p_ornaments', Dirichlet(0.5 * torch.ones(npcs, device=device)))
        #print(p_chordtones.shape)
        # we build a big categorical out of the chordtones and ornaments,
        # including notes of unknown type (marginalizing over the categories)
        #p_ct = p_is_chordtone_c       * p_chordtones
        #p_or = (1 - p_is_chordtone_c) * p_ornaments
        p_ct = torch.mm(torch.diag(p_is_chordtone_c), p_chordtones)
        p_or = torch.mm(torch.diag(1 - p_is_chordtone_c), p_ornaments)
        p_unobserved = p_ct + p_or
        p_tones = torch.cat([pobserve * p_ct, pobserve * p_or, (1-pobserve) * p_unobserved], dim=1)
    # distribution of note rate in chords
    rate_notes = pyro.sample(f'{pfx}rate_notes', Gamma(torch.tensor(3, device=device),torch.tensor(1, device=device)))
    
    # sampling the data:
    nchords = len(data['c'])
    subs = min(nchords,subsamples) if subsamples != None else None
    with pyro.plate(f'{pfx}data', nchords, subsample_size=subs) as ind:
        # pick a harmony
        c = pyro.sample(f'{pfx}c', Categorical(p_harmony), obs=data['c'][ind])
        # pick a number of notes
        nnotes = 1 + pyro.sample(f'{pfx}n', Poisson(rate_notes), obs=data['n'][ind]).int()
        # sample chordtones
        # Normally we would sample nnotes notes for each chord, but that doesn't work vectorized.
        # However, evaluating the probability ignores n, so we can just provide 1 here.
        notes = pyro.sample(f'{pfx}chord', Multinomial(1, p_tones[c], validate_args=False), obs=data['notes'][ind])
        chords = {'c': c,
                  'n': nnotes,
                  'counts': notes.reshape(-1,npcs)}
    return chords

## Guide

A simple guide that assumes the latent variables to be distributed independently.

In [ ]:
def chord_guide(npcs, nharmonies, cluster_assignment, data, pfx=None, subsamples=500, pobserve=0.5, init=dict(), optimize=True):
    if pfx is None or pfx == "":
        pfx = ""
    else:
        pfx += "_"
    nclusters = max(cluster_assignment) + 1
    
    # posterior of p_harmony
    params_p_harmony = init['harmonies'].to(device) if 'harmonies' in init else 0.5 * torch.ones(nharmonies, device=device)
    if optimize:
        params_p_harmony = pyro.param(f'{pfx}params_p_harmony',
                                      params_p_harmony,
                                      constraint=constraints.positive)
    pyro.sample(f'{pfx}p_harmony', Dirichlet(params_p_harmony))
    
    # posteriors of notes dists in harmonies (parameters)
    params_p_chordtones = init['chordtones'].to(device) if  'chordtones' in init else 0.5 * torch.ones(nharmonies, npcs, device=device)
    params_p_ornaments = init['ornaments'].to(device) if 'ornaments' in init else 0.5 * torch.ones(nharmonies, npcs, device=device)
    if optimize:
        params_p_chordtones = pyro.param(f'{pfx}params_p_chordtones',
                                         params_p_chordtones,
                                         constraint=constraints.positive)
        params_p_ornaments = pyro.param(f'{pfx}params_p_ornaments',
                                        params_p_ornaments,
                                        constraint=constraints.positive)
    
    # posterior of ornament probability (parameters)
    alpha_p_ict = init['is_ct'].to(device) if 'is_ct' in init else torch.ones(nclusters, device=device)
    beta_p_ict = init['is_or'].to(device) if 'is_or' in init else torch.ones(nclusters, device=device)
    if optimize:
        alpha_p_ict = pyro.param(f'{pfx}alpha_p_ict',
                                 alpha_p_ict,
                                 constraint=constraints.positive)
        beta_p_ict = pyro.param(f'{pfx}beta_p_ict',
                                beta_p_ict,
                                constraint=constraints.positive)
    
    with pyro.plate(f'{pfx}clusters', nclusters) as ind:
        pyro.sample(f'{pfx}p_is_chordtone', Beta(alpha_p_ict, beta_p_ict))
        
    # posteriors of ornament probability and note distributions
    with pyro.plate(f'{pfx}harmonies', nharmonies) as ind:
        pyro.sample(f'{pfx}p_chordtones', Dirichlet(params_p_chordtones[ind]))
        pyro.sample(f'{pfx}p_ornaments', Dirichlet(params_p_ornaments[ind]))
        
    #posterior of note rate
    alpha_rate_notes = init['sum_chords'].to(device) if 'sum_chords' in init else torch.tensor(3., device=device)
    beta_rate_notes = init['n_chords'].to(device) if 'n_chords' in init else torch.tensor(1., device=device)
    if optimize:
        alpha_rate_notes = pyro.param(f'{pfx}alpha_rate_notes',
                                      alpha_rate_notes,
                                      constraint=constraints.positive)
        beta_rate_notes = pyro.param(f'{pfx}beta_rate_notes',
                                     beta_rate_notes,
                                     constraint=constraints.positive)
    rate_notes = pyro.sample(f'{pfx}rate_notes', Gamma(alpha_rate_notes, beta_rate_notes))

In [ ]:
pyro.clear_param_store()
gc.collect()
chord_guide(3, 5, torch.tensor([0,1,2,3,4], device=device), None)

## Data and Conditioning

### Data Format

The input data (i.e. the observations that the model is conditioned on) is represented by three tensors:
- `c` for the chord labels (as "categorical" integers)
- `n` for the number of notes in each chord
- `notes` for the observed notes in each chord

Each of these tensors represents the values for all chords at the same time (i.e. a *vectorized* representation),
so the first dimension of each equals `nchords`, the number of chords.
`c` and `n` are vectors, i.e. their value for each chord is a scalar.
`notes` represents a vector for each chord that contains the counts of all pitch $\times$ note type pairs in the chord.
If we assume 29 pitch classes, we therefore have 87 entries: 29 for the chordtones, 29 for the ornaments, and 29 for the notes of unknown type.
As a result, `notes` has dimension `nchords` $\times$ 87.

The values of `c` represent each chord's type, which is distributed according to a categorical distribution.
In pyro/torch, categories are represented as integers, so we must convert textual labels into integers.
Similarly, the index of a note in `notes` is determined by it's pitch class and type (as outlined above).
While we allow negative pitch classes, they can be easily transformed into indices (and *vice versa*) by shifting all values by `npcs // 2`.

In [ ]:
def chord_tensor(notes):
    """Takes a list of notes as (fifth, type) pairs and returns a vector of counts."""
    notetype = {'chordtone': 0, 'ornament': 1, 'unknown': 2}
    chord = torch.zeros((3, npcs), device=device)
    for (fifth, t) in notes:
        chord[notetype[t], utils.fifth_to_index(fifth)] += 1
    return chord

def annot_data_obs(chords):
    """Helper function to turn a list of chord dictionary into a dictionary of observation vectors."""
    obs = {}
    obs["notes"] = torch.cat([chord_tensor(c['notes']).reshape((1,-1)) for c in chords], dim=0)
    obs["c"] = torch.tensor([c['label'] for c in chords], device=device)
    obs["n"] = torch.tensor([len(c['notes']) - 1. for c in chords], device=device)
    return obs

### Loading the Dataset

The data is loaded from a TSV file that.
The resulting dataframe is converted to the observation format that we pass to the model.

In [ ]:
chordtypes_common = np.array(
    ["major", "minor", "dominant-7th", "diminished",
     "full-diminished", "minor-7th", "half-diminished", "major-7th",
     "augmented", "minor-major-7th", "augmented-7th"],
    dtype="object")
dcml_chordtype_map = {
    "M": "major",
    "m": "minor",
    "Mm7": "dominant-7th",
    "o": "diminished",
    "o7": "full-diminished",
    "mm7": "minor-7th",
    "%7": "half-diminished",
    "MM7": "major-7th",
    "+": "augmented",
    "mM7": "minor-major-7th",
    "+7": "augmented-7th",
}
wiki_chordtype_map = {
    "major": "major",
    "minor": "minor",
    "dominant": "dominant-7th",
    "diminished": "diminished",
    "diminished-seventh": "full-diminished",
    "minor-seventh": "minor-7th",
    "half-diminished": "half-diminished",
    "major-seventh": "major-7th",
    "augmented": "augmented",
    "major-minor": "minor-major-7th",
    "augmented-seventh": "augmented-7th",
}

In [ ]:
def load_dataset(filename, alt_labels):
    print("loading dataset...") 
    df = utils.load_csv(filename)
    df = df[df.label.map(lambda l: l in alt_labels)]
    sizes = df.groupby(['chordid', 'label']).size()
    type_counts = sizes.groupby('label').size().sort_values(ascending=False)
    chordtypes = type_counts.index.tolist()
    df['numlabel'] = df.label.map(chordtypes.index)
    df['alt_label'] = df.label.map(alt_labels)
    
    # check if precomputed tensor data is available:
    prefn = filename + "_common_precomp.pt"
    if path.exists(prefn) and path.getmtime(prefn) > path.getmtime(filename):
        print("using precomputed tensor data.")
        obs = torch.load(prefn)
        obs["notes"] = obs["notes"].to(device)
        obs["c"] = obs["c"].to(device)
        obs["n"] = obs["n"].to(device)
        print(device)
        print([(k, v.device) for k, v in obs.items()])
    else:
        print('extracting chords...')
        chords = [{'label': label, 'notes': list(zip(grp.fifth, grp.type))}
                  for (_, label), grp in tqdm.tqdm(df.groupby(['chordid', 'numlabel']))]
        print('converting chords to tensors...')
        obs = annot_data_obs(chords)
        torch.save(obs, prefn)
    
    print(len(chordtypes), "chord types")
    print(len(obs["c"]), "chords")
    return df, obs, [alt_labels[l] for l in chordtypes]

In [ ]:
def clust_dist(X,Y):
    return 1 - (special.beta((X[0] + Y[0]) / 2, (X[1] + Y[1]) /2)) / math.sqrt(special.beta(X[0],X[1]) * special.beta(Y[0],Y[1]))

def mean_dist(X,Y):
    return abs(stats.beta.mean(X[0],X[1]) - stats.beta.mean(Y[0],Y[1]))

def kl_dist(X,Y):
    return []
        
#dist = mean_dist

def get_init_params(df, nharms, npcs):
    init = dict()
    
    init['harmonies'] = torch.tensor(df.groupby('numlabel').size().sort_values(ascending=False), device=device) + 0.5

    init['chordtones'] = torch.zeros([nharms,npcs], device=device) + 0.5
    for (numlabel, fifth), grp in df[df.type=='chordtone'].groupby(['numlabel','fifth']):
        init['chordtones'][numlabel, utils.fifth_to_index(fifth)] += grp.fifth.count()

    init['ornaments'] = torch.zeros([nharms,npcs], device=device) + 0.5
    for (numlabel, fifth), grp in df[df.type=='ornament'].groupby(['numlabel','fifth']):
        init['ornaments'][numlabel, utils.fifth_to_index(fifth)] += grp.fifth.count()
    
    init['is_ct'] = torch.tensor([sum(df[df.numlabel==l].type=='chordtone') for l in range(nharms)], device=device) + 1.
    init['is_or'] = torch.tensor([sum(df[df.numlabel==l].type=='ornament') for l in range(nharms)], device=device) + 1.
    
    chord_sizes = df.groupby('chordid').size()-1
    init['sum_chords'] = torch.tensor(sum(chord_sizes) + 3., device=device)
    init['n_chords'] = torch.tensor(len(chord_sizes) + 1., device=device)
    return init

After inferring the parameters we save them for easier inspection and reuse.

In [ ]:
def save_params(params, chordtypes, name):
    torch.save(params, name+'.pt')
    with open(name+'.json', 'w') as f:
        json.dump({'params': {key: val.tolist() for key,val in params.items()},
                   'chordtypes': chordtypes},
                  f)

## Inference

Inference of the posterior is done via variational inference, i.e. by optimizing the parameters of the guide.
The function `infer_posteriors` takes a dataset of observations,
performs the optimization, and returns the optimized parameters together with some of their histories.

In [ ]:
def infer_posteriors(obs, init, chordtypes, cluster_assignment,
                     nsteps=5_000, subsamples=10_000, particles=1,
                     plot_loss=True, save_as=None):
    # optimize the parameters of the guide
    pyro.clear_param_store()
    pyro.set_rng_seed(1625) # set every time for independent reproducibility
    svi = pyro.infer.SVI(model=chord_model,
                         guide=chord_guide,
                         optim=pyro.optim.Adam({"lr": 0.01, "betas": (0.95, 0.999)}),
                         #optim=pyro.optim.Adadelta({"lr": 1.0, "rho": 0.9}),
                         #optim=pyro.optim.SGD({"lr": 0.00005, "momentum": 0.9, "nesterov": True}),
                         loss=pyro.infer.Trace_ELBO(num_particles=particles))

    nharms = len(chordtypes)
    
    # set up histories for the loss and some of the parameters
    losses = np.zeros(nsteps)
    param_history = {name:np.zeros(nsteps) for name in ['alpha_rate_notes', 'beta_rate_notes']}#, 'alpha_p_ict', 'beta_p_ict']}
    root_history = np.zeros((nsteps,nharms))
    harm_history = np.zeros((nsteps,nharms))

    # run the optimization
    for i in tqdm.trange(nsteps):
        # update parameters and record loss
        losses[i] = svi.step(npcs, nharms, cluster_assignment, obs, subsamples=subsamples, init=init)
        
        # record values of some parameters
        ps = pyro.get_param_store()
        root_history[i] = ps.get_param('params_p_chordtones').cpu().detach()[:,fifth_range]
        harm_history[i] = ps.get_param('params_p_harmony').cpu().detach()
        for (name, value) in ps.items():
            if name in param_history:
                param_history[name][i] = value.cpu().item()

    # plot the loss
    if plot_loss:
        plt.figure()
        plt.plot(losses)
        plt.xlabel("iteration")
        plt.ylabel("loss")
        plt.show()
        print("loss variance (last 100 steps):", losses[-100:].var())
    
    params = dict((name, value.detach().cpu().detach().numpy()) for name, value in pyro.get_param_store().items())
    if save_as != None:
        save_params(params, chordtypes, save_as)
    
    return params, param_history, root_history, harm_history

To inspect the results and the behaviour of the optimization, we define some functions for plotting parameter histories and posterior distributions.

In [ ]:
def iterative_clustering(init, chordtypes, obs, name, dist=mean_dist, show_plots=True):
    nharms = len(chordtypes)

    # initial clustering
    cluster_assignment = dict((i,i) for i in range(nharms))

    # run iterative experiments
    outputs = []
    for it in range(nharms):
        nclusters = nharms - it
        print(f"iteration {it} ({nclusters} clusters).")
        print(cluster_assignment)
        
        # inference
        assignment = torch.tensor([cluster_assignment[i] for i in range(nharms)], device=device)
        params, hist, roots, harm = infer_posteriors(obs, init, chordtypes, assignment,
                                                     nsteps=350, subsamples=None, particles=1,
                                                     save_as="dcml_params"+str(nclusters),
                                                     plot_loss=show_plots)
        # record output
        outputs.append(dict({
            "params": params,
            "hist": hist,
            "roots": roots,
            "harm": harm,
            "cluster_assignment": cluster_assignment,
            "init": init,
        }))
        
        
        if show_plots:
            plot_p_ict(params, chordtypes, cluster_assignment, lower=0.7, upper=0.95)
        
        # compute next clustering / init
        if nclusters > 1:
            # find closest clusters
            alphas = params["alpha_p_ict"]
            betas = params["beta_p_ict"]
            dists = dict()
            for i in range(nclusters):
                for j in range(i+1, nclusters):
                    dists[(i,j)] = dist((alphas[i], betas[i]), (alphas[j], betas[j]))
            min1, min2 = min(dists, key=dists.get)

            # map clusters
            remaining = [i for i in range(nclusters) if i not in [min1,min2]]
            cluster_mapping = {**{min1: 0, min2: 0}, **dict((c,i+1) for i,c in enumerate(remaining))}
            
            # update assignment
            cluster_assignment = dict((h,cluster_mapping[c]) for h,c in cluster_assignment.items())

            # update init
            # ... with the posterior parameters from the previous run
            init = dict()
            init['harmonies'] = torch.tensor(params['params_p_harmony'], device=device)
            init['chordtones'] = torch.tensor(params['params_p_chordtones'], device=device)
            init['ornaments'] = torch.tensor(params['params_p_ornaments'], device=device)
            init['sum_chords'] = torch.tensor(params['alpha_rate_notes'], device=device)
            init['n_chords'] = torch.tensor(params['beta_rate_notes'], device=device)
            # ... and merged clusters
            init['is_ct'] = torch.zeros(nclusters-1, dtype=torch.float64, device=device)
            init['is_or'] = torch.zeros(nclusters-1, dtype=torch.float64, device=device)
            for i in range(nclusters-1):
                init['is_ct'][i] += alphas[cluster_mapping[i]]
                init['is_or'][i] += betas[cluster_mapping[i]]

    return outputs

## Bayes Factor Inference

In [ ]:
def bf_meta_model(npcs, nharms, cluster_assignments, data, **kwargs):
    nmodels = len(cluster_assignments)
    m = pyro.sample("model", Categorical(torch.ones(nmodels, device=device) / nmodels))
    return chord_model(npcs, nharms, cluster_assignments[m], data, **kwargs, pfx=f"m{m}")

def bf_meta_guide(npcs, nharmonies, cluster_assignments, data, inits, **kwargs):
    nmodels = len(cluster_assignments)
    params_model = pyro.param("params_model", torch.ones(nmodels, device=device) / nmodels, constraint=constraints.simplex)
    # pick a model uniformly, but assign the probability correctly using sample(obs=m)
    m = pyro.sample("model", Categorical(params_model))
    chord_guide(npcs, nharmonies, cluster_assignments[m], data,
                pfx=f"m{m}", init=inits[m], optimize=False, **kwargs)

In [ ]:
def bf_infer_model_posterior(obs, outputs, chordtypes,
                             nsteps=5_000, subsamples=10_000, particles=1,
                             plot_loss=True, save_as=None):
    # optimize the parameters of the guide
    pyro.clear_param_store()
    pyro.set_rng_seed(1625) # set every time for independent reproducibility
    svi = pyro.infer.SVI(model=bf_meta_model,
                         guide=bf_meta_guide,
                         optim=pyro.optim.Adam({"lr": 0.01, "betas": (0.95, 0.999)}),
                         #optim=pyro.optim.Adadelta({"lr": 1.0, "rho": 0.9}),
                         #optim=pyro.optim.SGD({"lr": 0.00005, "momentum": 0.9, "nesterov": True}),
                         loss=pyro.infer.Trace_ELBO(num_particles=particles))

    nharms = len(chordtypes)
    nmodels = len(outputs)
    assignments = [torch.tensor([outputs[m]['cluster_assignment'][i] for i in range(nharms)], device=device)
                           for m in range(nmodels)]
    inits = [outputs[i]['init'] for i in range(nmodels)]
    
    # set up histories for the loss and some of the parameters
    losses = np.zeros(nsteps)
    param_history = np.zeros((nsteps,nmodels))
    
    # run the optimization
    for i in tqdm.trange(nsteps):
        # update parameters and record loss
        losses[i] = svi.step(npcs, nharms, assignments, obs, inits=inits, subsamples=subsamples)
        
        # record values of some parameters
        ps = pyro.get_param_store()
        param_history[i] = ps.get_param(f'params_model').cpu().detach()

    # plot the loss
    if plot_loss:
        plt.figure()
        plt.plot(losses)
        plt.xlabel("iteration")
        plt.ylabel("loss")
        plt.show()
        print("loss variance (last 100 steps):", losses[-100:].var())
    
    params = dict((name, value.cpu().detach().numpy()) for name, value in pyro.get_param_store().items())
    if save_as != None:
        save_params(params, chordtypes, save_as)
    
    return params, param_history

In [ ]:
def bf_bayes_factors(params):
    params = params['params_model']
    nmodels = len(params)
    bfs = np.array([[params[i] / params[j] for j in range(nmodels)] for i in range(nmodels)])
    return bfs, params

def plot_bayes_factors(bfs, probs):
    nclusters = len(probs)
    labels = list(range(nclusters, 0, -1))

    sns.heatmap(bfs,
                xticklabels=labels, yticklabels=labels,
                norm=LogNorm(vmin=bfs.min(), vmax=bfs.max()), center=0)
    plt.show()
    sns.barplot(x=labels, y=probs)
    plt.show()

In [ ]:
def run_bayes_factors(obs, outputs, chordtypes, nsteps=500):
    params, hist = bf_infer_model_posterior(obs, outputs, chordtypes,
                                            nsteps=nsteps, subsamples=None, particles=1,
                                            plot_loss=True, save_as=None)
    return params, hist

## Plotting

In [ ]:
# histories

def plot_param_history(history):
    df = pd.DataFrame(history)
    df.plot()
    plt.xlabel("iteration")
    plt.show()
    
def plot_roots(root_history, ylabel='root parameters'):
    plt.plot(root_history)
    plt.xlabel("iteration")
    plt.ylabel(ylabel)
    plt.show()

def plot_histories(param_history, root_history, harm_history):
    plot_param_history(param_history)
    plot_roots(root_history)
    plot_roots(harm_history, ylabel='chord type parameters')

In [ ]:
# posteriors

# posterior of 'rate_notes'
def plot_note_rate(params, lower=0, upper=10):
    alpha = params['alpha_rate_notes']
    beta = params['beta_rate_notes']
    print(alpha)
    print(beta)
    x = np.linspace(lower, upper, 200)
    y = stats.gamma.pdf(x, alpha, scale=1/beta)
    plt.plot(x,y)
    plt.xlabel('rate_notes')
    plt.show()
    xrate = np.linspace(0,10,11)
    yrate = stats.nbinom.pmf(xrate, alpha, 1/(1+1/beta))
    plt.bar(xrate+1, yrate)
    plt.xlabel('nnotes')
    plt.show()
    
def plot_note_rates(phist, n=100, lower=0, upper=10):
    alphas = phist['alpha_rate_notes'][-n:]
    betas = phist['beta_rate_notes'][-n:]
    x = np.linspace(lower, upper, 200)
    ys = np.array([stats.gamma.pdf(x, a, scale=1/b) for (a,b) in zip(alphas,betas)]).transpose()
    plt.plot(x,ys, color='steelblue', alpha=0.5)
    plt.xlabel('rate_notes')
    plt.show()
    
# posterior of 'p_is_chordtone'
def plot_p_ict(params, harmtypes, cluster_assignment, lower=0, upper=1):
    alphas = params["alpha_p_ict"]
    betas  = params["beta_p_ict"]
    x = np.linspace(lower, upper, 200)
    y = np.array([stats.beta.pdf(x, a, b) for a, b in zip(alphas, betas)]).transpose()
    names = dict()
    for chord, cluster in cluster_assignment.items():
        name = harmtypes[chord]
        if(names.get(cluster) == None):
            names[cluster] = []
        names[cluster].append(name)
    # This might work
    its = list(names.items())
    its.sort()
    ns = [str.join(", ",it[1]) for it in its]
    plt.plot(x,y)
    plt.xlabel("p_is_chordtone")
    plt.legend(ns, bbox_to_anchor=(1., 1), loc='upper left')
    plt.show()

# posterior of chord type probabilities
def plot_chord_type_dist(params, labels):
    plt.figure(figsize=(6,9))
    alphas = params['params_p_harmony']
    plt.barh(np.arange(len(alphas)), alphas, tick_label=labels)
    plt.gca().invert_yaxis()
    plt.xlabel("params_p_harmony")
    plt.show()

# posteriors of note probabilities
def plot_chords(params, labels):
    post_chordtones = params['params_p_chordtones']
    post_ornaments = params['params_p_ornaments']
    for i, name in enumerate(labels):
        utils.plot_profile(post_chordtones[i], post_ornaments[i], name)
        utils.play_chord(post_chordtones[i])

# plot all posteriors
def plot_posteriors(params, chordtypes):
    plot_note_rate(params)
    plot_p_ict(params, chordtypes)
    plot_chord_type_dist(params, chordtypes)
    plot_chords(params, chordtypes)

## Clustering (DCML Corpus)

The DCML corpus is a collection of classical pieces with elaborate harmonic annotations.
Here we only distinguish the basic harmonic types defined in the annotation standard (triads and seventh chords),
since the extra information (inversion, suspensions, added notes etc.) do not change the type of the chord.

In [ ]:
# prepare the dataset
dcml_df, dcml_obs, dcml_chordtypes = load_dataset('data/dcml.tsv', dcml_chordtype_map)
gc.collect()
debug_tensors()

In [ ]:
dcml_init = get_init_params(dcml_df, len(dcml_chordtypes), npcs)
gc.collect()
debug_tensors()

In [ ]:
print(dcml_chordtypes)
#print(dcml_init)
#for init in dcml_inits:
#    print(init['cluster'])

In [ ]:
dcml_outputs = iterative_clustering(dcml_init, dcml_chordtypes, dcml_obs, "dcml")

In [ ]:
dcml_outputs[0]

In [ ]:
# plot the histories the parameters to check convergence
for output in dcml_outputs:
    plot_histories(output['hist'], output['roots'], output['harm'])

In [ ]:
for output in dcml_outputs:
    # plot the posterior distributions of the parameters
    #plot_note_rate(output['params'], lower=5.5, upper=5.65)
    #plot_note_rates(output['hist'], n=50, lower=5.5, upper=5.65)
    plot_p_ict(output['params'], dcml_chordtypes, output['cluster_assignment'], lower=0.7, upper=0.95)
    #plot_chord_type_dist(output['params'], dcml_chordtypes)
    #plot_chords(output['params'], dcml_chordtypes)

## Bayes Factor (DCML)

In [ ]:
dcml_cmp, dcml_cmp_hist = run_bayes_factors(dcml_obs, dcml_outputs, dcml_chordtypes, nsteps=1_000)

In [ ]:
plot_param_history(dcml_cmp_hist)

In [ ]:
dcml_bfs, dcml_m_params = bf_bayes_factors(dcml_cmp)
plot_bayes_factors(dcml_bfs, dcml_m_params)
dcml_bfs[8,0]

## Clustering (Wikifonia Corpus)

The Wikifonia dataset consists of leadsheets, i.e. melodies and chord labels.
It uses the chord types set in the MusicXML source of the chord-labels, which can be quite chaotic.
Therefore, we normalize the chord-types to the ones defined in the MusicXML standard,
removing unclear chord labels (which are rather rare).

In [ ]:
# prepare the dataset
wiki_df, wiki_obs, wiki_chordtypes = load_dataset('data/wikifonia.tsv', wiki_chordtype_map)
wiki_init = get_init_params(wiki_df, len(wiki_chordtypes), npcs)

In [ ]:
print(wiki_chordtypes)
#for init in wiki_inits:
    #print(init['cluster'])

In [ ]:
wiki_outputs = iterative_clustering(wiki_init, wiki_chordtypes, wiki_obs, "wiki")

In [ ]:
# plot the histories the parameters to check convergence
for output in wiki_outputs:
    plot_histories(output['hist'], output['roots'], output['harm'])
#plot_histories(whist, wroots, wharm)

In [ ]:
# plot the posterior distributions of the parameters
for output in wiki_outputs:
    # plot the posterior distributions of the parameters
    #plot_note_rate(output['params'], lower=2.35, upper=2.45)
    #plot_note_rates(output['hist'], n=50, lower=2.35, upper=2.45)
    plot_p_ict(output['params'], wiki_chordtypes, output['cluster_assignment'], lower=0.65, upper=0.95)
    #plot_chord_type_dist(output['params'], wiki_chordtypes)
    #plot_chords(output['params'], wiki_chordtypes)
#plot_note_rate(wiki_params, lower=2.35, upper=2.45)
#plot_note_rates(whist, n=100, lower=2.35, upper=2.45)
#plot_p_ict(wiki_params, wiki_chordtypes, lower=0.65, upper=0.95)
#plot_chord_type_dist(wiki_params, wiki_chordtypes)
#plot_chords(wiki_params, wiki_chordtypes)

## Bayes Factor (Wikifonia)

In [ ]:
wiki_cmp, wiki_cmp_hist = run_bayes_factors(wiki_obs, wiki_outputs, wiki_chordtypes, nsteps=1_000)

In [ ]:
plot_param_history(wiki_cmp_hist)

In [ ]:
wiki_bfs, wiki_m_params = bf_bayes_factors(wiki_cmp)
plot_bayes_factors(wiki_bfs, wiki_m_params)
wiki_bfs[8,0]

# Rest

In [ ]:
len(wiki_inits)

In [ ]:
wiki_outputs[0]['wiki_params']['params_cluster']

In [ ]:
wiki_inits[5]['cluster']

In [ ]:
dcml_outputs[0]['dcml_params']['params_cluster']

In [ ]:
dcml_inits[5]['cluster']

In [ ]:
dcml_inits[5]['is_ct']


In [ ]:
dcml_inits[5]['is_or']

In [ ]:
dcml_outputs[0]['dcml_params']['alpha_p_ict']

In [ ]:
dcml_outputs[0]['dcml_params']['beta_p_ict']

In [ ]:
stats.beta.mean(dcml_inits[5]['is_ct'][0],dcml_inits[5]['is_or'][0])

In [ ]:
stats.beta.mean(dcml_inits[5]['is_ct'][1],dcml_inits[5]['is_or'][1])

In [ ]:
stats.beta.mean(dcml_inits[5]['is_ct'][5],dcml_inits[5]['is_or'][5])

In [ ]:
stats.beta.mean(dcml_outputs[0]['dcml_params']['alpha_p_ict'][5],dcml_outputs[0]['dcml_params']['beta_p_ict'][5])

In [ ]:
stats.beta.mean(dcml_outputs[0]['dcml_params']['alpha_p_ict'][1],dcml_outputs[0]['dcml_params']['beta_p_ict'][1])

In [ ]:
stats.beta.mean(dcml_inits[13]['is_ct'][6],dcml_inits[13]['is_or'][6])

In [ ]:
stats.beta.mean(dcml_inits[13]['is_ct'][7],dcml_inits[13]['is_or'][7])

In [ ]:
stats.beta.mean(dcml_inits[13]['is_ct'][9],dcml_inits[13]['is_or'][9])

In [ ]:
stats.beta.mean(dcml_inits[13]['is_ct'][0],dcml_inits[13]['is_or'][0])

In [ ]:
stats.beta.mean(dcml_inits[13]['is_ct'][1],dcml_inits[13]['is_or'][1])